In [59]:
import json
import requests

In [60]:
def get_citsci_dataset_list(api="http://api.gbif.org/v1/",
                            namespace="citizenScience.mgrosjean.gbif.org"):
    '''
    Gets the dataset UUIDs for all the datasets currently tagged
    as scitizen science dataset.
    '''
    citizen_science_datasets = []
    offset = 0
    step = 900
    end_of_records = False
    while not end_of_records:
        param = {
            "offset": offset,
            "limit": step,
            "machineTagNamespace": namespace
        }
        # Query API
        response = requests.get(api + "dataset", param)
        if response.ok:
            citsci_dataset = response.json()
            for dataset in citsci_dataset["results"]:
                citizen_science_datasets.append(dataset["key"])
            offset += step
            end_of_records = citsci_dataset["endOfRecords"]
        else:
            print("ERROR")
            end_of_records = True
    return citizen_science_datasets

In [61]:
def create_download_given_query(login,
                                password,
                                download_query,
                                api="http://api.gbif.org/v1/"):
    '''
    Query the download API
    '''
    headers = {'Content-Type': 'application/json'}
    download_request = requests.post(api + "occurrence/download/request",
                                     data=json.dumps(download_query),
                                     auth=(login, password),
                                     headers=headers)
    if download_request.ok:
        print("ok")
    else:
        print(download_request)
    return download_request

In [63]:
# Build Download query

download_query = {}
download_query["creator"] = ""
download_query["notificationAddresses"] = [""]
download_query["sendNotification"] = True
download_query["format"] = "SIMPLE_CSV"

# Specify predicates
download_query["predicate"] = {
    "type": "and",
    "predicates": []
}
predicate_dataset = {
    "type": "in",
    "key": "DATASET_KEY",
    "values": get_citsci_dataset_list()
}
# I want only the insects from scitizen science dataset:
predicate_taxon = {
    "type": "equals",
    "key": "TAXON_KEY",
    "value": 216 # insect
}
download_query["predicate"]["predicates"].append(predicate_dataset)
download_query["predicate"]["predicates"].append(predicate_taxon)

In [64]:
login = ""
password = ""

In [ ]:
create_download_given_query(login, password, download_query)